In [ ]:
# Google Drive Mount

from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/Shareddrives/컨디션난조,분위기고조/텀프로젝트/NewsPolicyRecommendation/Data/PolicyData

/content/gdrive/Shareddrives/컨디션난조,분위기고조/텀프로젝트/NewsPolicyRecommendation/Data/PolicyData


In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 42 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 1s (7,121 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletyp

In [ ]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [787 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:13 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packag

bash: line 6: fg: no job control


In [ ]:
%ls

정책_2018_01.csv  정책_2019_02.csv  정책_2021_01.csv       정책_tfidf.csv
정책_2018_02.csv  정책_2020_01.csv  정책_2021_02.csv
정책_2019_01.csv  정책_2020_02.csv  정책_preprocessed.csv


In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ast
import re
from tqdm import tqdm
from konlpy.tag import Okt
from pathlib import Path
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
years = ['2018','2019','2020','2021']
periods = ['01','02']
name = ['상반기','하반기']

In [ ]:
data = pd.DataFrame()
for y in years:
  for p in periods:
    df = pd.read_csv('정책_{}_{}.csv'.format(y, p))
    df['year'] = y
    df['period'] = name[int(p)-1]
    data = pd.concat([data, df])
data.reset_index(drop=True, inplace=True)

In [ ]:
data.shape

(1311, 6)

In [ ]:
data.head()

,분야,번호,정책명,정책내용,year,period
0,금융/재정/조세,1,법정 최고금리 연 24%로 인하,"법정 최고금리 연 24%로 인하\n법무부 상사법무과 (02-2110-3256) , ...",2018,상반기
1,금융/재정/조세,2,전자어음 최장만기 단계적 단축,전자어음 최장만기 단계적 단축\n법무부 상사법무과 (02-2110-3256)\n분야...,2018,상반기
2,금융/재정/조세,3,신용카드를 활용한 본인확인서비스 신규 도입,신용카드를 활용한 본인확인서비스 신규 도입\n방송통신위원회 개인정보보호윤리과 (02...,2018,상반기
3,금융/재정/조세,4,외국인관광객 숙박요금 부가가치세 환급,외국인관광객 숙박요금 부가가치세 환급\n문화체육관광부 관광산업정책과 (044-203...,2018,상반기
4,금융/재정/조세,5,조달청 일반용역 적격심사시 최저임금 위반자 입찰평가 감점,조달청 일반용역 적격심사시 최저임금 위반자 입찰평가 감점\n조달청 기술서비스총괄과 ...,2018,상반기


In [ ]:
okt = Okt()
tqdm.pandas()
special = re.compile(r'[^ A-Za-z가-힣+]')
data['정책내용'] = data['정책내용'].apply(lambda x : special.sub(' ', str(x)))
data['정책내용'] = data['정책내용'].apply(lambda x : str(x).replace('\n',' '))
data['정책내용'] = data['정책내용'].apply(lambda x : str(x).replace('달라지는 정책 안내',' '))
data['정책내용'] = data['정책내용'].apply(lambda x : str(x).replace('달라지는 정책 개요',' '))
data['nouns'] = data['정책내용'].progress_apply(lambda x: ' '.join([i for i in okt.nouns(x) if len(i)!=1]))
data.to_csv('정책_preprocessed.csv', encoding = 'utf-8', index = False)

100%|██████████| 1311/1311 [00:47<00:00, 27.83it/s]


In [ ]:
data.head()

,분야,번호,정책명,정책내용,year,period,nouns
0,금융/재정/조세,1,법정 최고금리 연 24%로 인하,법정 최고금리 연 로 인하 법무부 상사법무과 금...,2018,상반기,법정 최고 금리 인하 법무부 상사 법무 금융위 서민 금융 분야 금융 재정 조세 대상...
1,금융/재정/조세,2,전자어음 최장만기 단계적 단축,전자어음 최장만기 단계적 단축 법무부 상사법무과 분야 금...,2018,상반기,전자 어음 만기 단계 단축 법무부 상사 법무 분야 금융 재정 조세 대상 중소 중견 ...
2,금융/재정/조세,3,신용카드를 활용한 본인확인서비스 신규 도입,신용카드를 활용한 본인확인서비스 신규 도입 방송통신위원회 개인정보보호윤리과 ...,2018,상반기,신용카드 활용 본인 확인 서비스 신규 도입 방송통신 위원회 개인 정보보호 윤리 분야...
3,금융/재정/조세,4,외국인관광객 숙박요금 부가가치세 환급,외국인관광객 숙박요금 부가가치세 환급 문화체육관광부 관광산업정책과 ...,2018,상반기,외국인 관광객 숙박 요금 부가가치세 환급 문화 체육관 광부 관광 산업 정책 분야 금...
4,금융/재정/조세,5,조달청 일반용역 적격심사시 최저임금 위반자 입찰평가 감점,조달청 일반용역 적격심사시 최저임금 위반자 입찰평가 감점 조달청 기술서비스총괄과 ...,2018,상반기,조달청 일반 용역 격심 사시 최저임금 위반 입찰 평가 감점 조달청 기술 서비스 총괄...


In [ ]:
data.tail()

,분야,번호,정책명,정책내용,year,period,nouns
1306,국토/교통,2,국내공항 짐배송서비스 시범사업 추진,국내공항 짐배송서비스 시범사업 추진 항공보안과 분야 국토...,2021,하반기,국내 공항 배송 서비스 시범 사업 추진 항공 보안 분야 국토 교통 대상 기타 관련 ...
1307,국토/교통,3,수도권에도 드론전용비행시험장이 운영됩니다,수도권에도 드론전용비행시험장이 운영됩니다 첨단항공과 분야...,2021,하반기,수도권 드론 전용 비행 시험 운영 첨단 항공 분야 국토 교통 대상 기타 관련 부처 ...
1308,국토/교통,4,항공 안전데이터 분석센터의 운영 시작,항공 안전데이터 분석센터의 운영 시작 항공안전정책과 분야...,2021,하반기,항공 안전 데이터 분석 센터 운영 시작 항공 안전 정책 분야 국토 교통 대상 기타 ...
1309,국토/교통,5,소화물배송대행서비스 사업자 인증제 시행,소화물배송대행서비스 사업자 인증제 시행 생활물류 긴급대응반 상황총괄대응과 ...,2021,하반기,화물 배송 대행 서비스 사업자 인증 시행 생활 물류 긴급 대응 상황 총괄 대응 분야...
1310,국토/교통,6,택배서비스사업 등록제 시행,택배서비스사업 등록제 시행 생활물류 긴급대응반 상황총괄대응과 ...,2021,하반기,택배 서비스 사업 등록 시행 생활 물류 긴급 대응 상황 총괄 대응 분야 국토 교통 ...


In [ ]:
data.shape

(1311, 7)

In [ ]:
corpora = data['nouns'].to_list()

In [ ]:
vectorizer = TfidfVectorizer(smooth_idf=True, use_idf=True)

In [ ]:
vectorizer.fit_transform(corpora)

<1311x7396 sparse matrix of type '<class 'numpy.float64'>'
	with 87098 stored elements in Compressed Sparse Row format>

In [ ]:
feature_names = vectorizer.get_feature_names()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


# 상위 N개 추출

In [ ]:
def sort_coo(coo_matrix):
    """Sort a dict with highest score"""
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature, score
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

# TF-IDF 활용하여 키워드 추출

In [ ]:
TOP_K_KEYWORDS = 10

In [ ]:
def get_keywords(vectorizer, feature_names, doc):
    """Return top k keywords from a doc using TF-IDF method"""

    #generate tf-idf for the given document
    tf_idf_vector = vectorizer.transform([doc])
    
    #sort the tf-idf vectors by descending order of scores
    sorted_items=sort_coo(tf_idf_vector.tocoo())

    #extract only TOP_K_KEYWORDS
    keywords=extract_topn_from_vector(feature_names,sorted_items,TOP_K_KEYWORDS)
    
    return list(keywords.keys())

# 정책 키워드 추출 후, 데이터프레임 CSV 저장

In [ ]:
result = []
for doc in corpora:
    df = {}
    df['full_text'] = doc
    df['top_keywords'] = get_keywords(vectorizer, feature_names, doc)
    result.append(df)
    
final = pd.DataFrame(result)
final

,full_text,top_keywords
0,법정 최고 금리 인하 법무부 상사 법무 금융위 서민 금융 분야 금융 재정 조세 대상...,"[금리, 최고, 법정, 인하, 이자제한법, 법무부, 사인, 부업, 대부, 금전]"
1,전자 어음 만기 단계 단축 법무부 상사 법무 분야 금융 재정 조세 대상 중소 중견 ...,"[어음, 단축, 만기, 전자, 폐해, 개월, 법무부, 발행, 단계, 공포]"
2,신용카드 활용 본인 확인 서비스 신규 도입 방송통신 위원회 개인 정보보호 윤리 분야...,"[본인, 신용카드, 확인, 신규, 수단, 활용, 도입, 휴대폰, 서비스, 방송통신]"
3,외국인 관광객 숙박 요금 부가가치세 환급 문화 체육관 광부 관광 산업 정책 분야 금...,"[숙박, 환급, 관광객, 외국인, 요금, 광부, 체육관, 문화, 부가가치세, 호텔]"
4,조달청 일반 용역 격심 사시 최저임금 위반 입찰 평가 감점 조달청 기술 서비스 총괄...,"[입찰, 조달청, 감점, 격심, 용역, 위반, 평가, 노동, 최저임금, 조달]"
...,...,...
1306,국내 공항 배송 서비스 시범 사업 추진 항공 보안 분야 국토 교통 대상 기타 관련 ...,"[공항, 배송, 출발, 서비스, 도착, 승객, 대행, 제주, 시범, 운영]"
1307,수도권 드론 전용 비행 시험 운영 첨단 항공 분야 국토 교통 대상 기타 관련 부처 ...,"[드론, 비행, 시험, 수도권, 전용, 구축, 연구, 화성, 시장, 준공]"
1308,항공 안전 데이터 분석 센터 운영 시작 항공 안전 정책 분야 국토 교통 대상 기타 ...,"[항공, 항공사, 데이터, 안전, 분석, 센터, 수집, 기관, 개별, 운영]"
1309,화물 배송 대행 서비스 사업자 인증 시행 생활 물류 긴급 대응 상황 총괄 대응 분야...,"[인증, 배송, 대행, 화물, 서비스, 물류, 생활, 사업자, 발전, 시행]"


In [ ]:
data.columns

Index(['분야', '번호', '정책명', '정책내용', 'year', 'period', 'nouns'], dtype='object')

In [ ]:
final = pd.concat([data[['분야', '번호', '정책명','year', 'period']], final], axis=1)

In [ ]:
final

,분야,번호,정책명,year,period,full_text,top_keywords
0,금융/재정/조세,1,법정 최고금리 연 24%로 인하,2018,상반기,법정 최고 금리 인하 법무부 상사 법무 금융위 서민 금융 분야 금융 재정 조세 대상...,"[금리, 최고, 법정, 인하, 이자제한법, 법무부, 사인, 부업, 대부, 금전]"
1,금융/재정/조세,2,전자어음 최장만기 단계적 단축,2018,상반기,전자 어음 만기 단계 단축 법무부 상사 법무 분야 금융 재정 조세 대상 중소 중견 ...,"[어음, 단축, 만기, 전자, 폐해, 개월, 법무부, 발행, 단계, 공포]"
2,금융/재정/조세,3,신용카드를 활용한 본인확인서비스 신규 도입,2018,상반기,신용카드 활용 본인 확인 서비스 신규 도입 방송통신 위원회 개인 정보보호 윤리 분야...,"[본인, 신용카드, 확인, 신규, 수단, 활용, 도입, 휴대폰, 서비스, 방송통신]"
3,금융/재정/조세,4,외국인관광객 숙박요금 부가가치세 환급,2018,상반기,외국인 관광객 숙박 요금 부가가치세 환급 문화 체육관 광부 관광 산업 정책 분야 금...,"[숙박, 환급, 관광객, 외국인, 요금, 광부, 체육관, 문화, 부가가치세, 호텔]"
4,금융/재정/조세,5,조달청 일반용역 적격심사시 최저임금 위반자 입찰평가 감점,2018,상반기,조달청 일반 용역 격심 사시 최저임금 위반 입찰 평가 감점 조달청 기술 서비스 총괄...,"[입찰, 조달청, 감점, 격심, 용역, 위반, 평가, 노동, 최저임금, 조달]"
...,...,...,...,...,...,...,...
1306,국토/교통,2,국내공항 짐배송서비스 시범사업 추진,2021,하반기,국내 공항 배송 서비스 시범 사업 추진 항공 보안 분야 국토 교통 대상 기타 관련 ...,"[공항, 배송, 출발, 서비스, 도착, 승객, 대행, 제주, 시범, 운영]"
1307,국토/교통,3,수도권에도 드론전용비행시험장이 운영됩니다,2021,하반기,수도권 드론 전용 비행 시험 운영 첨단 항공 분야 국토 교통 대상 기타 관련 부처 ...,"[드론, 비행, 시험, 수도권, 전용, 구축, 연구, 화성, 시장, 준공]"
1308,국토/교통,4,항공 안전데이터 분석센터의 운영 시작,2021,하반기,항공 안전 데이터 분석 센터 운영 시작 항공 안전 정책 분야 국토 교통 대상 기타 ...,"[항공, 항공사, 데이터, 안전, 분석, 센터, 수집, 기관, 개별, 운영]"
1309,국토/교통,5,소화물배송대행서비스 사업자 인증제 시행,2021,하반기,화물 배송 대행 서비스 사업자 인증 시행 생활 물류 긴급 대응 상황 총괄 대응 분야...,"[인증, 배송, 대행, 화물, 서비스, 물류, 생활, 사업자, 발전, 시행]"


In [ ]:
final.to_csv('정책 tfidf_0524.csv', encoding='utf-8', index=False)